In [1]:
import os
import torch
import torch.nn as nn
from torch.autograd import Variable
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim

Create dataset

In [2]:
trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (1.0,))])
# if not exist, download mnist dataset
INPUT_DATA_DIR = '/tmp/tensorflow/mnist/input_data'
train_set = dset.MNIST(root=INPUT_DATA_DIR, train=True, transform=trans, download=True)
test_set = dset.MNIST(root=INPUT_DATA_DIR, train=False, transform=trans, download=True)

In [3]:
batch_size = 64

train_loader = torch.utils.data.DataLoader(
                 dataset=train_set,
                 batch_size=batch_size,
                 shuffle=True)
test_loader = torch.utils.data.DataLoader(
                dataset=test_set,
                batch_size=batch_size,
                shuffle=False)

print('Training set size: {}'.format(len(train_loader)))
print('Test set size: {}'.format(len(test_loader)))


Training set size: 938
Test set size: 157


Build simple MLP

In [4]:
FEATURES_NUM = 28*28
CLASSES_NUM = 10

HIDDEN_FEATURES_1 = 256
HIDDEN_FEATURES_2 = 256

In [5]:
class MLPNet(nn.Module):
    def __init__(self):
        super(MLPNet, self).__init__()
        self.fc1 = nn.Linear(FEATURES_NUM, HIDDEN_FEATURES_1)
        self.fc2 = nn.Linear(HIDDEN_FEATURES_1, HIDDEN_FEATURES_2)
        self.fc3 = nn.Linear(HIDDEN_FEATURES_2, CLASSES_NUM)
    def forward(self, x):
        x = x.view(-1, FEATURES_NUM)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def name(self):
        return "MLP"

Create module, optimizer and loss

In [6]:
model = MLPNet()

In [7]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In [8]:
criterion = nn.CrossEntropyLoss()

Start learning:

In [9]:
MAX_STEP = 12#2000

In [10]:
import numpy as np

from tensorboardX import SummaryWriter
from tqdm import tqdm

writer = SummaryWriter(log_dir="/tmp/tensorflow/mnist/logs/mlp_torch")

for epoch in range(MAX_STEP):
    # trainning
    losses = []
    for batch_idx, (x, target) in enumerate(train_loader):
        optimizer.zero_grad()
        x, target = Variable(x), Variable(target)
        out = model(x)
        loss = criterion(out, target)
        losses.append(loss.data[0])
        loss.backward()
        optimizer.step()
        
    avg_loss = np.mean(losses)
    writer.add_scalar('train_loss', avg_loss, epoch)
    
    if epoch%10 == 0:
        print('epoch: {}, train loss: {:.6f}'.format(epoch, avg_loss))
        
    # testing
    correct_cnt = 0
    losses = []
    total_cnt = 0
    for batch_idx, (x, target) in enumerate(test_loader):
        x, target = Variable(x, volatile=True), Variable(target, volatile=True)
        
        out = model(x)
        loss = criterion(out, target)
        losses.append(loss.data[0])
        
        _, pred_label = torch.max(out.data, 1)
        total_cnt += x.data.size()[0]
        correct_cnt += (pred_label == target.data).sum()
    
    avg_loss = np.mean(losses)
    writer.add_scalar('test_loss', avg_loss, epoch)
    
    if epoch%10 == 0:
        print('epoch: {}, test loss: {:.6f}, acc: {:.3f}'.format(
            epoch, batch_idx+1, avg_loss, correct_cnt * 1.0 / total_cnt))
    
writer.close()

/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:16: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  app.launch_new_instance()


epoch: 0, train loss: 0.510895


/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:31: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:35: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


epoch: 0, test loss: 157.000000, acc: 0.243
epoch: 10, train loss: 0.038893
epoch: 10, test loss: 157.000000, acc: 0.070


Additional reading:
    - https://github.com/lanpa/tensorboardX
    - https://github.com/tensorflow/tensorboard